<a href="https://colab.research.google.com/github/cswcjt/Dacon_Bike/blob/main/bike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

import itertools
import os
import holidays

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [34]:
base_path = "/content/drive/MyDrive/fastcamp/datas/bike/"
train = pd.read_csv(base_path + "train.csv")
# test = pd.read_csv(base_path + "test.csv")
submission = pd.read_csv(base_path + "sample_submission.csv")

In [35]:
# train = train.rename(columns = {'일시':'date', '광진구':'gj', '동대문구':'dm','성동구':'sd', '중랑구':'jn'})
# train['date']= pd.to_datetime(train['date'], format='%Y%m%d')
train['일시'] = pd.to_datetime(train['일시'], format='%Y%m%d')
train

,일시,광진구,동대문구,성동구,중랑구
0,2018-01-01,0.592,0.368,0.580,0.162
1,2018-01-02,0.840,0.614,1.034,0.260
2,2018-01-03,0.828,0.576,0.952,0.288
3,2018-01-04,0.792,0.542,0.914,0.292
4,2018-01-05,0.818,0.602,0.994,0.308
...,...,...,...,...,...
1456,2021-12-27,3.830,3.416,2.908,2.350
1457,2021-12-28,4.510,3.890,3.714,2.700
1458,2021-12-29,4.490,3.524,3.660,2.524
1459,2021-12-30,4.444,3.574,3.530,2.506


In [36]:
# regions = ['gj', 'dm', 'sd', 'jn']
regions = ['광진구', '동대문구', '성동구', '중랑구']

In [38]:
def predict(train, column):
    model = Prophet()
    df = pd.DataFrame()
    df['ds'] = train['일시']
    df['y'] = train[column]
    model.fit(df)
    future = model.make_future_dataframe(periods=334, freq='D')
    forecast = model.predict(future)
    return forecast.loc[1461:, 'yhat'].values

for region in regions:
    submission[region] = predict(train, region)
   
submission

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpuq988z08/gs6ns6x5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpuq988z08/qntv4i_y.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=38819', 'data', 'file=/tmp/tmpuq988z08/gs6ns6x5.json', 'init=/tmp/tmpuq988z08/qntv4i_y.json', 'output', 'file=/tmp/tmpuq988z08/prophet_modeldcwfpq6e/prophet_model-20221201122600.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:26:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:26:00 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpuq9

,일시,광진구,동대문구,성동구,중랑구
0,20220101,5.110203,4.662176,3.896330,3.303345
1,20220102,4.821968,4.401556,3.769503,3.082229
2,20220103,5.121813,5.151970,4.121842,3.323606
3,20220104,5.240828,5.303184,4.215095,3.448420
4,20220105,5.470450,5.442060,4.370883,3.569264
...,...,...,...,...,...
329,20221126,8.962125,8.007408,6.372682,5.352272
330,20221127,8.575407,7.668120,6.167637,5.070159
331,20221128,8.775184,8.337838,6.441324,5.250297
332,20221129,8.792578,8.406381,6.455402,5.313642
